In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import mannwhitneyu
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter, FuncFormatter

import statsmodels.formula.api as smf

import scipy.stats as stats

sns.set_theme(style="whitegrid")

# Ice Fishing Fleet Visualizations


In [ ]:
landings = pd.read_csv("../data/derived/landings_cleaned.csv", low_memory=False)

In [ ]:
# ax = landings.groupby(by=['seasonal_year']).vessel_type.value_counts(normalize=True).unstack().plot.bar(stacked=True, color={'ice': '#b0dac2', 'water': '#436b88'})
# ax.set_ylabel('Share of landings (%)')
# ax.set_xlabel('Seasonal Year (August-August)')
# ax.legend = plt.legend()
# ax.legend.set_title('Type of Fishing')

In [ ]:
ax = (
    landings.groupby(by=["seasonal_year", "vessel_type"])
    .amount_in_kg.sum()
    .unstack()
    .apply(lambda x: x / sum(x), axis="columns")
    .plot(kind="bar", stacked=True, color={"ice": "#b0dac2", "water": "#436b88"})
)
ax.set_ylabel("Share of the total catch (%)")
ax.set_xlabel("Seasonal Year (August-August)")
ax.legend = plt.legend()
ax.legend.set_title("Type of Fishing")

# change axes to 0-100
# show entire fishing industry as the reference.. maybe second viz look just at nearshore and just halibut

In [ ]:
ax = (
    landings.query('vessel in ["Sled", "Snowmobile"]')
    .groupby(by=["seasonal_year", "vessel"])
    .amount_in_kg.sum()
    .unstack()
    .apply(lambda x: x / sum(x), axis="columns")
    .plot(kind="bar", stacked=True, color={"Sled": "#e49b0f", "Snowmobile": "#b22222"})
)
ax.set_ylabel("Share of the total catch (%)")
ax.set_xlabel("Seasonal Year (August-August)")
ax.legend = plt.legend()
ax.legend.set_title("Vessel Type")

In [ ]:
landings["year_month"] = landings.sales_date.apply(lambda x: "-".join(x.split("-")[:2]))

In [ ]:
# # Reorder the 'vessel' groups
# landings['vessel'] = pd.Categorical(landings['vessel'],
#                                    categories=['Larger Inshore Vessel', 'Dinghy', 'Sled', 'Snowmobile'],
#                                    ordered=True)

# # Create a stacked bar plot with custom colors and reordered categories
# ax = landings.groupby(by=['seasonal_year', 'vessel']).size().unstack().plot.bar(
#     stacked=True, figsize=(10, 10),
#     color={'Sled': '#e49b0f', 'Snowmobile': '#b22222', 'Dinghy': '#436b88', 'Larger Inshore Vessel': '#104e8b'})

# ax.set_ylabel("Number of landings")
# ax.set_xlabel("Month - Year")
# ax.legend(title="Vessel Type")
# plt.legend(bbox_to_anchor=(1, .05), loc='lower right')

In [ ]:
ax = (
    landings.query(
        'vessel in ["Sled", "Snowmobile", "Dinghy", "Larger Inshore Vessel"]'
    )
    .groupby(by=["seasonal_year", "vessel"])
    .amount_in_kg.sum()
    .unstack()
    .plot(
        kind="bar",
        stacked=False,
        figsize=(10, 5),
        color={
            "Sled": "#e49b0f",
            "Snowmobile": "#b22222",
            "Dinghy": "#436b88",
            "Larger Inshore Vessel": "#104e8b",
        },
    )
)
ax.set_ylabel("Total catch [Million kg]")
ax.set_xlabel("Fishing Season (August - August)")
ax.legend(title="Vessel Type", fontsize="8")


def millions_formatter(x, pos):
    return f"{x / 1e6:.0f} "


ax.yaxis.set_major_formatter(FuncFormatter(millions_formatter))

# stacked bar chart hard to read when they do not add up to total
# grouped bar chart
# test change

### Economic


In [ ]:
# fishers = landings.groupby(['seasonal_year', 'vessel_type']).seller_id.nunique().unstack()
# ax = fishers.plot(kind='bar', stacked=True, color={'ice': '#b0dac2', 'water': '#436b88'})
# ax.set_ylabel('Count of Fishers')
# ax.set_xlabel('Fishing Season (August-August)')
# ax.legend(title="Fishing Type")


# fishers = pd.melt(frame=fishers, ignore_index=False, value_vars=['ice', 'water'], value_name='num_fishers')

In [ ]:
# revenue = landings.groupby(['seasonal_year', 'vessel_type']).value.sum().unstack()
# ax = revenue.plot(kind='bar', stacked=True, color={'ice': '#b0dac2', 'water': '#436b88'})
# ax.set_ylabel('Gross Revenue [Million DKK]')
# ax.set_xlabel('Fishing Season (August-August)')
# ax.legend(title="Fishing Type")

# def millions_formatter(x, pos):
#      return f'{x / 1e6:.0f} '

# ax.yaxis.set_major_formatter(FuncFormatter(millions_formatter))

In [ ]:
fisher_types = pd.read_csv("../data/derived/fishertypes.csv")

In [ ]:
landings = landings.merge(
    fisher_types.drop(columns="ice_proportion_of_income"),
    on=["seller_id", "seasonal_year"],
)

In [ ]:
income = (
    landings.groupby(["seasonal_year", "seller_id"])
    .value.sum()
    .rename("income")
    .reset_index()
)
income = income.merge(
    fisher_types.drop(columns="ice_proportion_of_income"),
    on=["seller_id", "seasonal_year"],
)
median_income = income.groupby(["seasonal_year", "type"]).income.median()
median_income = median_income.rename("median_income").reset_index()

In [ ]:
# Set custom colors for 'ice' and 'water'
custom_palette = {
    "ice-only": "#b0dac2",
    "majority-ice": "#f59e42",
    "majority-water": "#8c199e",
    "water-only": "#436b88",
}

ax = sns.lineplot(
    data=median_income,
    x="seasonal_year",
    y="median_income",
    hue="type",
    palette=custom_palette,
)
ax.set_xlabel("Ice Fishing Season (August-August)")
ax.set_ylabel("Median Earnings Per Capita (Thousand DKK)")
ax.legend(title="Fisher Type")


def thousands_formatter(x, pos):
    return f"{x / 1e3:.0f} "


ax.yaxis.set_major_formatter(FuncFormatter(thousands_formatter))

In [ ]:
income_by_locality = (
    landings.groupby(["seasonal_year", "seller_id", "sellers_locality"])
    .value.sum()
    .rename("income")
    .reset_index()
)
income_by_locality = income_by_locality.merge(
    fisher_types.drop(columns="ice_proportion_of_income"),
    on=["seller_id", "seasonal_year"],
)
median_income_by_locality = income_by_locality.groupby(
    [
        "seasonal_year",
        "sellers_locality",
        "type",
    ]
).income.median()
median_income_by_locality = median_income_by_locality.rename(
    "median_income"
).reset_index()
median_income_by_locality

In [ ]:
family_income = pd.read_csv("../data/raw/family-income.csv", na_values="...")

family_income = family_income[["locality", "time", "family type", "Family income"]]
family_income = family_income.rename(
    columns={
        "locality": "sellers_locality",
        "time": "seasonal_year",
        "family type": "family_type",
        "Family income": "family_income",
    }
)
family_income = family_income.pivot(
    index=["seasonal_year", "sellers_locality"],
    columns="family_type",
    values="family_income",
)


median_income_by_locality = median_income_by_locality.pivot(
    index=["seasonal_year", "sellers_locality"], columns="type", values="median_income"
)

median_income_by_locality = median_income_by_locality.merge(
    family_income, left_index=True, right_index=True
)
median_income_by_locality.to_csv("../data/derived/median_income_by_locality.csv")

# TODO: Make a large table (for the supplemental materials) that shows the difference between each median income of each fisher type and the reference median income for each family type

In [ ]:
median_income_by_locality = median_income_by_locality.loc[
    2022,
    [
        "Single people, total",
        "Couples, total",
        "Families, total",
        "ice-only",
        "majority-ice",
    ],
]

In [ ]:
median_income_by_locality = median_income_by_locality.dropna(
    subset=["ice-only", "majority-ice"], how="all"
)

In [ ]:
median_income_by_locality["Maximum allowed income, ice-only"] = (
    median_income_by_locality["ice-only"] * 2
)
median_income_by_locality["Maximum allowed income, majority-ice"] = (
    median_income_by_locality["majority-ice"] * 2
)

In [ ]:
median_income_by_locality.loc[:, "Maximum income per day, ice-only"] = (
    median_income_by_locality["Maximum allowed income, ice-only"] / 365
)
median_income_by_locality.loc[:, "Maximum income per day, majority-ice"] = (
    median_income_by_locality["Maximum allowed income, majority-ice"] / 365
)

In [ ]:
median_income_by_locality = median_income_by_locality[
    [
        "Single people, total",
        "Couples, total",
        "Families, total",
        "ice-only",
        "Maximum allowed income, ice-only",
        "Maximum income per day, ice-only",
        "majority-ice",
        "Maximum allowed income, majority-ice",
        "Maximum income per day, majority-ice",
    ]
]

In [ ]:
localities = pd.read_csv("../resources/localities.csv")
localities = localities[["locality", "geo_lat"]]

In [ ]:
median_income_by_locality = (
    median_income_by_locality.merge(
        right=localities, left_index=True, right_on="locality"
    )
    .set_index("locality")
    .sort_values(by="geo_lat", ascending=False)
    .drop(columns="geo_lat")
)

In [ ]:
median_income_by_locality.to_csv("../data/derived/median_income_by_locality.csv")

In [ ]:
# # Set custom colors for 'ice' and 'water'
# custom_palette = {'ice': '#b0dac2', 'water': '#436b88'}

# median_earnings_per_capita = pd.melt(frame=landings.groupby(['seasonal_year', 'vessel_type', 'seller_id']).amount_in_kg.sum().unstack(1).groupby('seasonal_year').median(), ignore_index=False, value_name='median_catch_per_capita', value_vars=['ice', 'water'])
# median_earnings_per_capita.reset_index(inplace=True)
# ax = sns.lineplot(data=median_earnings_per_capita, x='seasonal_year', y='median_catch_per_capita', hue='vessel_type', palette=custom_palette)
# ax.set_xlabel("Ice Fishing Season (August-August)")
# ax.set_ylabel("Median Catch (kg) Per Capita")
# ax.legend(title="Fishing Type")

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

median_amount = (
    landings.groupby(by=["year_month", "vessel_type"])
    .amount_in_kg.median()
    .unstack()
    .reset_index()
)

median_amount.year_month = pd.to_datetime(median_amount.year_month, format="%Y-%m")
median_amount.set_index("year_month", inplace=True)

median_amount.fillna(0, inplace=True)
result = seasonal_decompose(median_amount["ice"], model="additive")
fig = result.plot(resid=False)
fig.axes[0].set_ylabel("Median catch [kg]")

median_amount_decomposed = pd.DataFrame()
median_amount_decomposed["trend"] = result.trend
median_amount_decomposed["seasonal"] = result.seasonal
median_amount_decomposed["vessel_type"] = "ice"
median_amount_decomposed

In [ ]:
result.trend.value_counts()

# Ice Fishing Season Visualizations


In [ ]:
ppk_locality = pd.read_csv("../data/derived/ppk_locality.csv", low_memory=False)

ax = sns.boxplot(
    data=ppk_locality, x="seasonal_year", y="ppk", legend=True, hue="vessel_type"
)

ax.set_xlabel("Ice Fishing Season (August-August)")
ax.set_ylabel("Greenland halibut price paid to fishers (DKK / kg)")

In [ ]:
(
    ppk_locality.set_index("sellers_locality")
    .groupby("seasonal_year")
    .ppk.idxmax()
    .rename("Locality with the maximum PPK")
)

In [ ]:
(
    ppk_locality.set_index("sellers_locality")
    .groupby("seasonal_year")
    .ppk.idxmin()
    .rename("Locality with the minimum PPK")
)

## Total Daily Catch During Fishing Seasons 2012-2022


In [ ]:
total_daily_catch = pd.read_csv("../data/derived/total_daily_catch.csv")
ax = sns.lineplot(
    total_daily_catch,
    x="seasonal_day",
    y="cumulative",
    hue="seasonal_year",
    palette="colorblind",
)

ax.set_xlabel("Ice Fishing Season, Days from 1 August")
ax.set_ylabel("Cumulative Catch (kg)")
ax.legend(title="Ice Fishing Season")

### Correlation Tests (Season Start, End, and Length ~ Fishing Season)


In [ ]:
# total_first_catch = pd.read_csv("../data/derived/total_first_catch.csv")
# total_first_catch.corr(method="kendall").loc["first_catch_day", "season"]

In [ ]:
# ax = sns.lmplot(data=total_first_catch, x="season", y="first_catch_day", legend=False)

# ax.set_axis_labels("Ice Fishing Season (August - August)", "First Day of Ice Fishing in Greenland")

In [ ]:
# total_last_catch = pd.read_csv("../data/derived/total_last_catch.csv")
# total_last_catch.corr(method="kendall").loc["last_catch_day", "season"]

In [ ]:
# ax = sns.lmplot(data=total_last_catch, x="season", y="last_catch_day", legend=False)

# ax.set_axis_labels("Ice Fishing Season (August - August)", "Last Day of Ice Fishing in Greenland")

In [ ]:
total_ice_season_length = pd.read_csv("../data/derived/total_ice_season_length.csv")
total_ice_season_length.corr(method="kendall").loc["ice_season_length", "season"]

In [ ]:
total_ice_season_length

In [ ]:
# ax = sns.lmplot(
#     data=total_ice_season_length, x="season", y="ice_season_length", legend=False
# )

# ax.set_axis_labels("Ice Fishing Season (August - August)", "Length of Ice Fishing Season (Days)")

### Local First Catch


In [ ]:
locality_first_catch = pd.read_csv("../data/derived/locality_first_catch.csv")

ax = sns.lineplot(
    data=locality_first_catch.groupby(by=["season", "sellers_locality"])
    .first_catch_day.min()
    .reset_index(),
    x="season",
    y="first_catch_day",
    legend=False,
)
plt.xticks(range(2012, 2023))  # Adjust the range based on your data
ax.set_xlabel("Ice Fishing Season (August - August)")
ax.set_ylabel("First Day of Ice Fishing")

### Local Last Catch


In [ ]:
locality_last_catch = pd.read_csv("../data/derived/locality_last_catch.csv")

ax = sns.lineplot(
    data=locality_last_catch.groupby(by=["season", "sellers_locality"])
    .last_catch_day.max()
    .reset_index(),
    x="season",
    y="last_catch_day",
    legend=False,
)
plt.xticks(range(2012, 2023))  # Adjust the range based on your data

ax.set_xlabel("Ice Fishing Season (August - August)")
ax.set_ylabel("Last Day of Ice Fishing")

### Ice Fishing Catch


In [ ]:
ice_landings = landings.query('vessel_type == "ice"')

ax = sns.lineplot(
    data=ice_landings.groupby(by=["seasonal_year", "sellers_locality"])
    .amount_in_kg.sum()
    .reset_index(),
    x="seasonal_year",
    y="amount_in_kg",
    legend=False,
)
plt.xticks(range(2012, 2023))  # Adjust the range based on your data

ax.set_xlabel("Ice Fishing Season (August - August)")
ax.set_ylabel("Catch (tonnes)")


def millions_formatter(x, pos):
    return f"{x / 1e3:.0f} "


ax.yaxis.set_major_formatter(FuncFormatter(millions_formatter))

### Number of Fields Fished, 2012-2022


In [ ]:
num_fields = pd.read_csv("../data/derived/total_num_fields.csv")
# ax = sns.barplot(data=num_fields, x="season", y="n_fields", color="#b0dac2")


# ax.set_xlabel("Ice Fishing Season (August - August)")
# ax.set_ylabel("Count of Ice Fishing Areas Fished")

### Local Number of Fields fished, 2012-2022

#todo create visualization of this, and stats test


In [ ]:
locality_num_fields = pd.read_csv("../data/derived/locality_num_fields.csv")
locality_num_fields = locality_num_fields.pivot(
    index="sellers_locality", columns="seasonal_year", values="n_fields"
).fillna(0)

# Normalize each locality to maximum number of field codes
locality_num_fields = locality_num_fields.div(
    locality_num_fields.max(axis="columns"), axis=0
)

locality_num_fields = (
    locality_num_fields.merge(right=localities, left_index=True, right_on="locality")
    .set_index("locality")
    .sort_values(by="geo_lat", ascending=False)
    .drop(columns="geo_lat")
)
plt.figure(figsize=(8, 12))
custom_cmap = sns.color_palette("Blues")
ax = sns.heatmap(locality_num_fields, cmap=custom_cmap)

ax.set_xlabel("Ice Fishing Season (August - August)")
ax.set_ylabel("Localities with Ice Fishing Activities, by Descending Latitude (°N)")
# plt.text(0.5, -0.09, 'Heat Map of Field Codes Fished, by locality, by year, normalized by each locality to Maximum Number of Field Codes', fontsize=10, ha='center', va='center', transform=plt.gca().transAxes)

## Ice Fishing Season Trend


In [ ]:
ice_season_trend = pd.read_csv("../data/derived/ice_season_trend.csv")

ax = sns.histplot(ice_season_trend, x="duration_change", color="#b0dac2")

ax.set_xlabel("Ice Fishing Season Trend (days)")
ax.set_ylabel("Count")

In [ ]:
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")
g = sns.catplot(
    ice_season_trend.dropna().sort_values(by="duration_change"),
    kind="strip",
    x="duration_change",
    y="sellers_locality",
    height=6,
    aspect=0.8,
    linewidth=1,
    edgecolor="w",
    orient="h",
    jitter=False,
)

sns.despine(left=True, bottom=True)
g.axes.flat[0].yaxis.grid(True)
g.axes.flat[0].xaxis.grid(True)
g.axes.flat[0].set_xlim([-15, 35])
g.axes.flat[0].set_xticks(range(-15, 40, 5))
plt.axvline(0)
g.set_xlabels("Ice season duration trend in days")
g.set_ylabels("Locality")

In [ ]:
ax = sns.scatterplot(ice_season_trend, x="ice_season_trend", y="lat")

ax.set_xlim([-10, 10])
plt.xticks(range(-10, 10))

ax.set_xlabel("Ice Fishing Season Trend (days)")
ax.set_ylabel("Latitude (°N)")

In [ ]:
# ax = sns.lineplot(
#     locality_seasons[locality_seasons.first_or_last == "last"],
#     x="season",
#     y="amount_in_kg",
# )
# plt.xticks(range(2012, 2022))  # Adjust the range based on your data
# ax.set_xlabel("Ice Fishing Season (August - August)")
# ax.set_ylabel("Catch (kg)")

In [ ]:
locality_seasons = pd.read_csv("../data/derived/locality_seasons.csv")

locality_seasons["landing_date"] = pd.to_datetime(locality_seasons["landing_date"])

# Filter the dataframe to include only rows where 'first_or_last' is 'last'
filtered_locality_seasons = locality_seasons[
    locality_seasons["first_or_last"] == "first"
]

# Extract month and day components from 'landing_date'
filtered_locality_seasons["landing_month"] = filtered_locality_seasons[
    "landing_date"
].dt.month
filtered_locality_seasons["landing_day"] = filtered_locality_seasons[
    "landing_date"
].dt.day

# Find the earliest month and day
earliest_month = filtered_locality_seasons["landing_month"].min()
earliest_day = filtered_locality_seasons["landing_day"].min()

print("Earliest Month:", earliest_month)
print("Earliest Day:", earliest_day)